In [ ]:
# -*- coding: utf-8 -*-
# ---
# jupyter:
#   jupytext:
#     text_representation:
#       extension: .py
#       format_name: light
#       format_version: '1.5'
#       jupytext_version: 1.14.0
#   kernelspec:
#     display_name: Python 3 (ipykernel)
#     language: python
#     name: python3
# ---

# # 5. Thử nghiệm Xử lý Hình thái

# Mục tiêu: Áp dụng các phép toán hình thái để tinh chỉnh kết quả phân vùng (ảnh nhị phân/mask).

# ## Import thư viện
import os
import sys
import cv2
import numpy as np

# Thêm thư mục src vào sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src import utils
from src import segmentation # Để tạo mask ban đầu
from src import morphology
from src import restoration # Có thể cần làm mịn ảnh gốc trước



In [ ]:
# ## Tải ảnh mẫu và tạo mask ban đầu
image_filename = 'sample_sign.jpg' # Đảm bảo ảnh này tồn tại
image_path = os.path.join('..', 'data', 'raw', image_filename) 
img_original = utils.load_image(image_path, cv2.IMREAD_COLOR)

if img_original is not None:
    img_smoothed = restoration.apply_gaussian_blur(img_original, kernel_size=(3, 3))
    
    # Tạo mask nhị phân ban đầu (ví dụ bằng Otsu)
    initial_mask = segmentation.otsu_threshold_segmentation(img_smoothed)
    
    # Hoặc tạo mask dựa trên màu (thử với màu xanh dương)
    # lower_blue = np.array([100, 150, 50]) 
    # upper_blue = np.array([140, 255, 255])
    # initial_mask = segmentation.color_threshold_segmentation(img_smoothed, lower_blue, upper_blue)

    # ## Áp dụng các phép toán hình thái
    
    # Kernel thường dùng
    kernel_3x3 = (3, 3)
    kernel_5x5 = (5, 5)

    # 1. Phép Co (Erosion) - làm mỏng, loại bỏ điểm nhỏ
    eroded_mask = morphology.apply_erosion(initial_mask, kernel_size=kernel_3x3, iterations=1)

    # 2. Phép Giãn (Dilation) - làm dày, nối liền
    dilated_mask = morphology.apply_dilation(initial_mask, kernel_size=kernel_3x3, iterations=1)

    # 3. Phép Mở (Opening) - loại bỏ nhiễu trắng nhỏ (salt noise)
    opened_mask = morphology.apply_opening(initial_mask, kernel_size=kernel_5x5)

    # 4. Phép Đóng (Closing) - lấp lỗ đen nhỏ (pepper noise)
    closed_mask = morphology.apply_closing(initial_mask, kernel_size=kernel_5x5)

    # ## Hiển thị kết quả
    utils.display_images(
        ["Mask Gốc", "Sau Co", "Sau Giãn", "Sau Mở", "Sau Đóng"],
        [initial_mask, eroded_mask, dilated_mask, opened_mask, closed_mask],
        figsize=(25, 5)
    )
    
    # Thử kết hợp Mở -> Đóng để làm sạch nhiễu và lấp lỗ
    cleaned_mask = morphology.apply_opening(initial_mask, kernel_size=(3,3))
    final_mask = morphology.apply_closing(cleaned_mask, kernel_size=(7,7))
    utils.display_images(
        ["Mask Gốc", "Sau Mở (3x3)", "Sau Mở rồi Đóng (7x7)"],
        [initial_mask, cleaned_mask, final_mask],
        figsize=(18, 6)
    )
    
else:
    print(f"Lỗi: Không thể tải ảnh tại '{image_path}'.")